## Client Side VoWiFi Evaluation

The files that have been used to conduct our client side analysis are included as a zip file
`dumps.zip` (use git lfs to clone this repository)

The extraction step may take a few minutes!

In [ ]:
# General Settings

# Will extract the zipfile to the current folder inside the repository
DUMPS_FOLDER="./"

# Enables autoreload extension to reload files on imports
#%load_ext autoreload
#%autoreload 2

In [ ]:
# MAY TAKE A FEW MINUTES (many small files ~2mins)
# Extract the files included in the zip file to dumps
import zipfile
import os

# Define the path to the zip file
zip_file_path = 'dumps.zip'

# Create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of the zip file to the specified directory
    zip_ref.extractall(DUMPS_FOLDER)

print("Unzipping complete!")

In [ ]:
# FOLDER AND FILE STRUCTURE(included in dumps.zip)

## APPLE
#Folders
APPLE_FOLDER=DUMPS_FOLDER+"dumps/20240201_Apple/"
APPLE_FOLDER_DATA=DUMPS_FOLDER+"/dumps/20240201_Apple/data/"
APPLE_EVALUATION_FOLDER="evaluation/APPLE/"

#Files
APPLE_IPCC_URLS=APPLE_FOLDER_DATA+"ipcc_urls.txt"
APPLE_IPCC_CARRIERS_REPRODUCE=APPLE_FOLDER+"/ipcc_carriers_reproduce.txt"
APPLE_IPPC_EVALUATION_RESULTS_REPRODUCE=APPLE_EVALUATION_FOLDER+"apple_ike_configuration_parameters_reproduce.json"
APPLE_IPPC_UPDATE_TIMES_REPRODUCE=APPLE_EVALUATION_FOLDER+"config_dirs_update_time_reproduce.json"
APPLE_IPCC_FILES=APPLE_FOLDER+"data/"

## SAMSUNG
#Folders
SAMSUNG_FOLDER=DUMPS_FOLDER+"dumps/Samsung_Clientside/"
SAMSUNG_EVALUATION_FOLDER="evaluation/SAMSUNG/"
#Files
SAMSUNG_APN_FILE=SAMSUNG_FOLDER+"epdg_apns_conf.xml"
SAMSUNG_APN_EVALUATION_RESULTS_REPRODUCE=SAMSUNG_EVALUATION_FOLDER+"samsung_ikev2_conf_reproduce.json"

## XIAOMI
#Folders
XIAOMI_FOLDER=DUMPS_FOLDER+"dumps/20230822_Xiaomi_13_Pro_0A_SM8550-AB/"
MBN_EVALUATION_FOLDER="evaluation/MBN/"

#Files
XAOMI_EVALUATION_RESULTS_REPRODUCE=MBN_EVALUATION_FOLDER+"mbn_xiaomi_configuration_parameters_reproduce.json"


## OPPO
#Folders
OPPO_FOLDER=DUMPS_FOLDER+"dumps/20231206_Oppo_Find_X6_Pro_5G_0A_SM8550-AB/"

#Files
OPPO_EVALUATION_RESULTS_REPRODUCE=MBN_EVALUATION_FOLDER+"mbn_oppo_ikev2_configuration_parameters_reproduce.json"



### Apple

Apple uses iOS Carrier Profiles (IPCC) to configure iPhones for mobile networks. We relied on ipcc-downloader (https://github.com/mrlnc/ipcc-downloader) to extract the IPCC bundles. The python3-script is included in the repository under client-side/ipcc-downloader.

Dumps.zip includes the downloaded IPCC carrier profiles and the ipcc_urls.txt which is the log of the URLS where IPCC bundles were downloaded from. 


In [ ]:
# Filter the ipcc_urls.txt for URLS that include iPhone IPCC Bundles and store carrier names

with open(APPLE_IPCC_URLS, 'r') as file:
    lines = file.readlines()

# Filter Providers with 'iphone' IPCCS (case insensitive)
iphone_ipcc_carriers = sorted(list(set([line.split("/")[-1].strip() for line in lines if 'iphone' in line.split("/")[-1].lower()])),key=str.casefold)

with open(APPLE_IPCC_CARRIERS_REPRODUCE,'w') as f_ipcc_carriers:
    for entry in iphone_ipcc_carriers:
        f_ipcc_carriers.write(entry+"\n")

In [ ]:
# Perform the evaluation of the APPLE IPCC Bundles
from extract_apple_ikev2_configuration_parameters import *

evaluate_apple_ipccs(APPLE_FOLDER_DATA,APPLE_IPCC_CARRIERS_REPRODUCE,APPLE_IPPC_EVALUATION_RESULTS_REPRODUCE,APPLE_IPPC_UPDATE_TIMES_REPRODUCE)

In [ ]:
from evaluation.APPLE.count_apple_json import *

apple_statistics=count_apple(APPLE_IPPC_EVALUATION_RESULTS_REPRODUCE)

### SAMSUNG



In [ ]:
from extract_samsung_ikev2_configuration_parameters import *

evaluate_samsung_apn_file(SAMSUNG_APN_FILE,SAMSUNG_APN_EVALUATION_RESULTS_REPRODUCE)

In [ ]:
from evaluation.SAMSUNG.count_samsung_json import *

samsung_statistics=count_samsung_apn_file(SAMSUNG_APN_EVALUATION_RESULTS_REPRODUCE)

### Xiaomi 

MBN Files

In [ ]:
from extract_mbn_ikev2_configuration_parameters import *

evaluate_mbn_files(XIAOMI_FOLDER,XAOMI_EVALUATION_RESULTS_REPRODUCE)

In [ ]:
from evaluation.MBN.count_mbn_json import *


xiaomi_statistics=count_mbn_results(XAOMI_EVALUATION_RESULTS_REPRODUCE)

### Oppo

MBN Files

In [ ]:
from extract_mbn_ikev2_configuration_parameters import *

evaluate_mbn_files(OPPO_FOLDER,OPPO_EVALUATION_RESULTS_REPRODUCE)

In [ ]:
from evaluation.MBN.count_mbn_json import *


oppo_statistics=count_mbn_results(OPPO_EVALUATION_RESULTS_REPRODUCE)

### Visualizations

#### DH Groups



In [ ]:
#Files 
DH_BARS_REPRODUCE="visualizations/dh_bars/ike_dh_groups_reproduce.pdf"
DEPRECATED_REPRODUCE="visualizations/deprecated/deprecated_bars_reproduce.pdf"
REKEY_REPRODUCE="visualizations/rekey/cdf_rekey_hours_reproduce.pdf"


In [ ]:
#from visualizations.dh_bars.dh_bars import *
ikev2_dh_group_data = {
        'Apple': {"dh_groups":apple_statistics['parsed_ikev2_params']['ikev2_dh_group_list'],"set":apple_statistics['ikev2_params']['ikev2_dh_group_list']},
        'Xiaomi': {"dh_groups":xiaomi_statistics['parsed_ikev2_params']['ikev2_dh_group_list'],"set":xiaomi_statistics['ikev2_params']['ikev2_dh_group_list']},
        'Oppo':{"dh_groups":oppo_statistics['parsed_ikev2_params']['ikev2_dh_group_list'],"set":oppo_statistics['ikev2_params']['ikev2_dh_group_list']},
        'Samsung': {"dh_groups":samsung_statistics['parsed_ikev2_params']['ikev2_dh_group_list'],"set":samsung_statistics['ikev2_params']['ikev2_dh_group_list']}
    }
ikev2_dh_group_data 

In [ ]:
from visualizations.dh_bars.dh_bars import *

visualize_dh_groups(ikev2_dh_group_data,DH_BARS_REPRODUCE)

#### Deprecated Algorithms




In [ ]:
from visualizations.deprecated.deprecated_bars import *

deprecated_ike_parameters = {
    "Apple":apple_statistics["deprecated"],
    "Xiaomi":xiaomi_statistics["deprecated"],
    "Oppo":oppo_statistics["deprecated"],
    "Samsung":samsung_statistics["deprecated"]
}

deprecated_ike_parameters



In [ ]:
visualize_deprecated_bars(deprecated_ike_parameters,DEPRECATED_REPRODUCE)

#### CDF Rekey Timers

In [ ]:
from visualizations.rekey.cdf_rekey_hours import *

rekey_dict = {
    "Apple" : apple_statistics['parsed_ikev2_params']['ikev2_lifetime'],
    "Xiaomi" : xiaomi_statistics['parsed_ikev2_params']['ikev2_sa_rekey_timer_soft_sec'],
    "Oppo" : oppo_statistics['parsed_ikev2_params']['ikev2_sa_rekey_timer_soft_sec'],
    "Samsung" : samsung_statistics['parsed_ikev2_params']['ikev2_lifetime']
    
}

rekey_dict

In [ ]:
rekey_cdf(rekey_dict,REKEY_REPRODUCE)